# Making Pseudocode For Eligability Rules Using LLM Agents

**Rationale:** 

There is a large time lag between policy making plain text documents and programmers turning these into progamable rules. One issue might be that these two teams speak different "languages" so translation and compromise takes time.

**Hypothesis:** 

Bridging the gap between programmer and policy through introduction of a mutally understood intermediate step will allow both parties to collaborate and problem solve faster.

**Approach:**

Use LLM User Agents to turn policy into pseudocode and have other agents adopt the personas of policy and programmers to critique the pseudocode until it is reflective and easily readable.

## Feedback

**RPA:**

- Initial demo of problem being solved **isn't** an issue seen by the RPA
- Pulling out eligability rules for an experienced RPA person doesn't take very long
- Highlighting areas of solid wording in the rules text would be highly beneficial
- Can we categorise the different types of rules (Eligability vs Checking Rules)

**Policy (Not Content Writers):**
- Initial demo of problem being solved **is** an issue seen by Policy
- Suggesting which ALBs (and area they care about) would be interesting

**Additional Thoughts after Demos:**
- Can I pull out some summary info?
- Can I pull out some conversation points?


In [4]:
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd
import autogen
import yaml
import os

config_file = json.loads(
    str('''{"model": "gpt-4",
        "api_key": "APIKEYHERE"}'''))

with open('OAI_CONFIG_LIST', 'w') as fp:
    json.dump([config_file], fp)

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    # filter_dict={
    #     "model": ["gpt-3.5-turbo"],
    },
)

gpt4_config = {
    "seed": 1337,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
}

In [22]:
user_proxy = autogen.UserProxyAgent(
    name="admin",
    system_message=f"""You coordinate the tasks.

    You take a user input which is a plain text document written in prose. 
    And run it through the below workflow with the aim of SNL formatted text representing the eligability rules within the document.

    Workflow:
    - Extract the eligability rules from the text using eligability_extractor
    - Convert the eligability rules into a controlled natural language format for easy interpretation of rules, done by "CNL_generator"
    - Quality check whether the controlled natural language statements meets the "policy_expert" expectations and collect feedback. 
    - If feedback is recieved from "policy_expert". Give the feedback to the "CNL_generator" for ammendments and repeat the process
    - Convert the controlled natural language formatted rules in a structured natural language format to start to represent these rules as statements, done by "SNL_generator"
    - Convert the structured natural language formatted statements into pseudocode functions, done by "pseudocode_generator".


    Once you are happy with the output - submit "TERMINATE" into the chain. Only do this once a file has been saved.
    """,
    human_input_mode = "NEVER",
    llm_config=gpt4_config,
    code_execution_config=False,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

eligability_extractor = autogen.AssistantAgent(
    name="eligability_extractor",
    llm_config=gpt4_config,
    system_message='''You take a document outlining a Defra policy for a land action which a person can apply for and get money. 
    Part of the process is assessing someones eligability for the policy - the eligability criteria is within the document. 
    your role is to export the section or sections of the document containing the eligability criteria.



    When producing the answers please try and ensure you are quoting from the policy rather than rewording the document in any way.
''',
)

CNL_generator = autogen.AssistantAgent(
    name="CNL_generator",
    llm_config=gpt4_config,
    system_message=''''You are responsible for converting the eligability rules from the "eligability_extractor" and writing them in a "Controlled Natural Language" format.

    You will take each numbered point and separately turn them to Controlled Natural Language statments.


    The output should be easy to understand by everyone
    ''',
)

policy_expert = autogen.AssistantAgent(
    name="policy_expert",
    llm_config=gpt4_config,
    system_message=''''You are highly skilled document writer who specialises in creating Defra policy documents like the one inputted.
    You are well versed in writing legal style documents but are not technical in that you cannot program computers or write code.

    Your role in the workflow is to compare the Controlled Natural Language statements from "CNL_generator" to the text submitted by the user.

    Your will feedback on:
    - Whether you think each Controlled Natural Language statement is missing key information from the input text from the user
    - Whether each Controlled Natural Language statement is understandable given you cannot read or write code

    If required you should suggest that this feedback be taken into account by "CNL_generator" who would regenerate thier response

    Once you are happy with an answer - tell the admin so they can carry on the workflow with the "SNL_generator"

    ''',
)


SNL_generator = autogen.AssistantAgent(
    name="SNL_generator",
    llm_config=gpt4_config,
    system_message=''''You are responsible for converting the controlled natural language statements from the "CNL_generator" AND the eligability criteria from "eligability_extractor" 
    then rewrite each of these points in a "Structured Natural Language" format.

    You will take each numbered point and separately turn them to Structured Natural Language statments, using bullet points to represent each sub statement.


    The output should be easy to understand by everyone
    ''',
)

## Agent that: pulls out the first line of each statement. The bits that end in :
### analyse these to show what common statements are, and say which bits of info we are interested in. 

pseudocode_generator = autogen.AssistantAgent(
    name="pseudocode_generator",
    llm_config=gpt4_config,
    system_message=''''You are responsible for converting the Structured Natural Language statments from the "SNL_generator" and turning them into pseudocode functions.

    You will take each numbered point and separately turn them into pseudocode functions. Print the SNL statement as the header for each function.

    ''',
)




groupchat = autogen.GroupChat(agents=[user_proxy, 
                                      eligability_extractor,
                                      CNL_generator,
                                      policy_expert,
                                      SNL_generator,
                                      pseudocode_generator],
                              messages=[],
                              admin_name = user_proxy,
                              max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)


policy_content = '''This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. This means you can take soil from each land parcel and combine it into a single sample to test the SOM. If you need to take soil samples on areas within a land parcel containing historic or archaeological features, including scheduled monuments, you must follow the requirements explained in section 5.6: ‘Land with historic or archaeological features’ of the SFI scheme guidance. To help improve national data on the condition of England’s soils, Defra or the RPA may ask you to provide some information about your soil assessment and SOM testing results during this action’s 3-year duration. This data is one of the public goods that this action is paying for and part of wider government measures in relation to our soils policy and strategy. We’ll publish guidance on GOV.UK about what information is needed and how to supply it. For the soil assessment and soil management plan, you must: For the SOM testing, each year of this action’s 3-year duration you must: If the timing meets the requirements above, you can do the SOM testing at the same time as your other routine soil testing required in the farming rules for water. If you enter land into this action which has not been tested for SOM within the last 5 years, you should try to do the SOM testing on that land during the first 12 months of your SFI agreement. If the laboratory that tests your soil samples does not have capacity to do the tests during the first 12 months of this action’s duration, you must: It’s up to you how you do this action, as long as you: Advice to help you do this action will be published before applications are fully launched from summer 2024. It will not be part of this action’s requirements. You must keep: You must supply this evidence when we ask for it. You can do the following actions or options on the same area in a land parcel as this action. Some actions or options can only be done on the same area if they’re done at a different time of year to this action. For example, winter cover followed by a summer companion crop. Read ‘What to do’ and ‘When to do it’ to find out when this action must be done. If an action or option cannot be located on the same area, you may be able to do it on a different area in the same land parcel. Read section 6. ‘Eligible land in other funding schemes’ for more information. You can do the following actions or options on the eligible boundaries of a land parcel entered into this action: Don’t include personal or financial information like your National Insurance number or credit card details.'''
user_proxy.initiate_chat(
    manager,
    message=f"""run the worklow and output questions eligability criteria for the following policy content: {policy_content}'''
    """,
)

admin (to chat_manager):

run the worklow and output questions eligability criteria for the following policy content: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar

ChatResult(chat_id=None, chat_history=[{'content': "run the worklow and output questions eligability criteria for the following policy content: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and 

In [ ]:
# script_writer = autogen.AssistantAgent(
#     name="script_writer",
#     llm_config=gpt4_config,
#     system_message='''You are the data exporter and run code.

#     You take the final output from the admin after it has been checked and convert it into a pandas dataframe then export it to a csv file.
#     You write python code accomplish these tasks.
#     Wrap the code in a code block that specifies the script type.
#     The user can't modify your code.
#     So do not suggest incomplete code which requires others to modify.
#     Don't use a code block if it's not intended to be executed by the executor.
#     Don't include multiple code blocks in one response.
#     Do not ask others to copy and paste the result.
#     Don't out elements like 'And so on' write the script in it's entirety.
#     Check the execution result returned by the executor.
#     If the result indicates there is an error, fix the error and output the code again.
#     Suggest the full code instead of partial code or code changes.
#     If the error can't be fixed or if the task is not solved even after the code is executed successfully,
#     analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.



#     Save the output as 'how_questions.csv' and overwrite any other files with that name.
# ''',
# )

# executor = autogen.UserProxyAgent(
#     name="Executor",
#     system_message="Executor. Execute the code written by the script_writer and report the result.",
#     human_input_mode="NEVER",
#     code_execution_config={"last_n_messages": 3, "work_dir": "titanic", "use_docker": False},

# )